In [1]:
import asyncio
from uuid import uuid4

In [2]:
from pydantic import BaseModel

In [3]:
from matter_persistence.redis.manager import CacheManager

In [4]:
# example DTO
class Test(BaseModel):
    some_field: int
    some_other_field: list[int]

# test type
t = Test(some_field=0, some_other_field=[_ for _ in range(5)])

In [5]:
# random organisation and internal id
ORGANISATION_ID = uuid4()
INTERNAL_ID = uuid4()

In [6]:
cache_manager = CacheManager(host="localhost", port=6379) # make sure there is a redis instance running!

In [7]:
# check if cache is alive
async def check():
    return await cache_manager.is_cache_alive()

await check()

True

In [8]:
# save pydantic type to cache
async def save():
    return await cache_manager.save_pydantic_object(ORGANISATION_ID, INTERNAL_ID, t, Test)

await save()

In [9]:
# check if pydantic object exists in cache
async def exists():
    return await cache_manager.cache_record_exists(ORGANISATION_ID, INTERNAL_ID, Test)

await exists()

1

In [10]:
# find pydantic object in cache
async def find():
    return await cache_manager.find_pydantic_object(ORGANISATION_ID, INTERNAL_ID, Test)

await find()

CacheRecordModel(internal_id='5b4cd28b-bee9-4983-aefe-34f74bc01ee2', hash_key='3ec94434-d8ca-404b-a901-5b68bafd8e50_Test_43f2681cb4828fa1c65c26ffd22c33c3e2e043f6', organization_id=UUID('3ec94434-d8ca-404b-a901-5b68bafd8e50'), value={'some_field': 0, 'some_other_field': [0, 1, 2, 3, 4]}, expiration=None)

In [11]:
# delete pydantic object from cache
async def delete():
    return await cache_manager.delete_pydantic_object(ORGANISATION_ID, INTERNAL_ID, Test)

await delete()

In [12]:
# make sure pydantic object is deleted from cache
await exists()

0

In [13]:
await find() # make sure returns exception

CacheRecordNotFoundError: 